# Download CPJUMP1 Data

This notebook documents the workflow for downloading and processing data from the JUMP Cell Painting dataset, available in the Cell Painting Gallery.

We focus on datasets where cells have been perturbed by overexpression of genes using open reading frame (ORF) vectors, which artificially increase the production of specific proteins.

Key steps in this workflow:
- Load configuration and metadata from a YAML file.
- Filter experimental metadata to include only plates with ORF perturbations.
- Download each plate's data as a CSV file, convert it to Parquet format, and save it in the `./data` directory.

Each individual plate data file is saved as a `parquet` file in the `./data` folder. If a file with the same name already exists, it will be replaced with the newly downloaded data.

In [1]:
import sys
import pathlib
import polars
import time
import tqdm

sys.path.append("../../")
from utils import io_utils

Setting parameters for the notebook:

- `pert_type (str)`: Perturbation of interest

In [2]:
pert_type = "orf"

Setting input and output paths

In [3]:
# setting config path
config_path = pathlib.Path("../nb-configs.yaml").resolve(strict=True)

# setting results setting a data directory
data_dir = pathlib.Path("./data").resolve()
data_dir.mkdir(exist_ok=True)

# creating a metadata directory
metadata_dir = (data_dir / "metadata").resolve()
metadata_dir.mkdir(exist_ok=True)

# creating a platemaps directory in the metadata directory
platemap_dir = (metadata_dir / "platemaps").resolve()
platemap_dir.mkdir(exist_ok=True)

Downloading the experimental metadata file and saving it into the `metadata` directory

In [4]:
# loading config file and setting experimental metadata URL
nb_configs = io_utils.load_configs(config_path)
CPJUMP1_exp_metadata_url = nb_configs["links"]["CPJUMP1-experimental-metadata-source"]

# read in the experimental metadata CSV file and only filter down to plays that
# have an ORF perturbation
exp_metadata = polars.read_csv(
    CPJUMP1_exp_metadata_url, separator="\t", has_header=True, encoding="utf-8"
)

# filtering the metadata to only includes plates that their perturbation types are orfs
exp_metadata = exp_metadata.filter(exp_metadata["Perturbation"].str.contains("orf"))

# save the experimental metadata as a csv file
exp_metadata.write_csv(metadata_dir / "CPJUMP1-experimental-metadata.tsv", separator="\t")

# display
exp_metadata.head()

Batch,Plate_Map_Name,Assay_Plate_Barcode,Perturbation,Cell_type,Time,Density,Antibiotics,Cell_line,Time_delay,Times_imaged,Anomaly,Number_of_images
str,str,str,str,str,i64,i64,str,str,str,i64,str,i64
"""2020_11_04_CPJUMP1""","""JUMP-Target-1_orf_platemap""","""BR00117006""","""orf""","""A549""",96,100,"""absent""","""Parental""","""Day0""",1,"""Phalloidin""",27648
"""2020_11_04_CPJUMP1""","""JUMP-Target-1_orf_platemap""","""BR00117020""","""orf""","""A549""",48,100,"""absent""","""Parental""","""Day0""",1,"""none""",27648
"""2020_11_04_CPJUMP1""","""JUMP-Target-1_orf_platemap""","""BR00117021""","""orf""","""A549""",48,100,"""absent""","""Parental""","""Day0""",1,"""none""",27648
"""2020_11_04_CPJUMP1""","""JUMP-Target-1_orf_platemap""","""BR00117022""","""orf""","""U2OS""",48,100,"""absent""","""Parental""","""Day0""",1,"""none""",27648
"""2020_11_04_CPJUMP1""","""JUMP-Target-1_orf_platemap""","""BR00117023""","""orf""","""U2OS""",48,100,"""absent""","""Parental""","""Day0""",1,"""none""",27648


Downloading the aggregated profiles that pertains to the selected `pert_type`

In [ ]:
# setting CPJUMP1 source link, this points to the main directory where all the plate data
# is stored
header_link = nb_configs["links"]["CPJUMP1-source"]

# create a for loop with progress bar for downloading plate data
for plate in tqdm.tqdm(
    exp_metadata["Assay_Plate_Barcode"].to_list(), desc="Downloading plates"
):
    # constructing the plate data source URL
    plate_data_source = f"{header_link}/{plate}/{plate}_normalized_negcon.csv.gz"

    # reading the plate data from the source URL
    # if the plate cannot be downloaded and read, it will skip to the next plate
    try:
        orf_plate_df = polars.read_csv(plate_data_source, separator=",", has_header=True)
    except polars.errors.ReadError:
        print(f"Failed to download and read plate data for {plate}. Skipping...")
        continue

    # saving the plate data to a parquet file
    orf_plate_df.write_parquet(data_dir / f"{plate}_normalized_negcon.parquet")

    # sleep to avoid overwhelming the AWS hosting the data
    time.sleep(1)

Downloading the platemaps associated with the selected `pert_type`

In [5]:
# downloading the plate maps
pert_platemap_url = nb_configs["links"]["CPJUMP1-platemaps-source"]
pert_platemap_name = f"JUMP-Target-1_{pert_type}_platemap.txt"

# constructing the full URL for the plate map
platemap_url = f"{pert_platemap_url}/{pert_platemap_name}"

# downloading the plate map
platemap_df = polars.read_csv(platemap_url, separator="\t")

# save it to the platemap directory
platemap_df.write_csv(platemap_dir / pert_platemap_name)

# display the first few rows of the plate map
platemap_df.head()

well_position,broad_sample
str,str
"""A01""","""ccsbBroad304_00900"""
"""A02""","""ccsbBroad304_07795"""
"""A03""","""ccsbBroad304_02826"""
"""A04""","""ccsbBroad304_01492"""
"""A05""","""ccsbBroad304_00691"""
